In [1]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score
import lightgbm as lgb
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import label_binarize
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score
from tqdm import tqdm_notebook
import joblib
import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv("../../2-数据转换/data_nor.csv")
data_columns = pd.read_csv("../../3-特征选取/data_columns.csv")
columns = data_columns.columns.values
columns = np.append(columns, "risk")
data = data[columns]
category_columns = [ col for col in data.columns if data[col].dtype == 'object' ]
print(len(category_columns))
# LabelEncoder
for col in tqdm_notebook(data[category_columns]):
    encoder = LabelEncoder()
    encoder.fit(list(data[col].values))
    data[col] = encoder.transform(list(data[col].values))
print(data)

4


  0%|          | 0/72550 [00:00<?, ?it/s]

        city   latitude   longitude  attacktype1_txt  targsubtype1_txt  \
0       4035  37.005105  -89.176269                0                73   
1      15269  37.791927 -122.225906                2                18   
2      12390  43.076592  -89.412488                3                50   
3      12390  43.072950  -89.386694                3                27   
4       5528  39.758968 -104.876305                3                50   
...      ...        ...         ...              ...               ...   
72545    219  12.849085   45.037275                2                35   
72546   3154  28.709444   82.163611                3                28   
72547  17467  15.305307   43.019490                2                32   
72548   9328  34.523842   69.140304                0                83   
72549  21608  37.688889  -97.336111                1                71   

       weapsubtype1_txt  date  risk  
0                    27     4     1  
1                    26     5     1

In [3]:
number_columns = [ col for col in data.columns if data[col].dtype != 'object' ]
number_columns.remove("risk")
#min-max
for col in number_columns:
    data[col] = (data[col] - data[col].min()) / (data[col].max() - data[col].min())
print(data)

           city  latitude  longitude  attacktype1_txt  targsubtype1_txt  \
0      0.181904  0.681564   0.204755            0.000          0.657658   
1      0.688351  0.688223   0.106546            0.250          0.162162   
2      0.558561  0.732945   0.204053            0.375          0.450450   
3      0.558561  0.732914   0.204130            0.375          0.243243   
4      0.249211  0.704869   0.158102            0.375          0.450450   
...         ...       ...        ...              ...               ...   
72545  0.009873  0.477141   0.603580            0.250          0.315315   
72546  0.142187  0.611361   0.713903            0.375          0.252252   
72547  0.787440  0.497927   0.597584            0.250          0.288288   
72548  0.420521  0.660566   0.675203            0.000          0.747748   
72549  0.974123  0.687351   0.180508            0.125          0.639640   

       weapsubtype1_txt      date  risk  
0              0.931034  0.500000     1  
1              

In [4]:
data.to_csv('data_stand.csv', index=False, encoding='utf_8_sig')

In [5]:
data = pd.read_csv("data_stand.csv")
data.shape
X = data.drop(columns=['risk'], axis=1)
y = data['risk']
kf = KFold(n_splits=10, shuffle=True, random_state=0)
splits = kf.split(X, y)
next(iter(splits))

(array([    0,     1,     2, ..., 72547, 72548, 72549]),
 array([    3,    14,    18, ..., 72525, 72542, 72544]))

In [6]:
#LR
from sklearn.linear_model import LogisticRegression

lr_roc_scores = []
lr_precision_scores = []
lr_recall_scores = []
lr_f1_scores = []
lr_acc_scores = []
lr_feature_importances = pd.DataFrame(index=None)
lr_feature_importances['features'] = data.drop(['risk'], axis=1).columns

for k, (train_indices, test_indices) in enumerate(splits):
    print("第 %d 折\n" % (k + 1))
    X_train, X_test = X.iloc[train_indices], X.iloc[test_indices]
    y_train, y_test = y.iloc[train_indices], y.iloc[test_indices]
    
    labels = [0, 1, 2, 3]
    y_one_hot = label_binarize(y_test, classes=labels)
    LR = LogisticRegression(random_state=0)
    LR.fit(X_train, y_train)
#     lr_feature_importances[f'fold_{k+1}'] = LR.feature_importance
    y_pred_prob =LR.predict_proba(X_test)
    y_pred=LR.predict(X_test)
    
    acc = accuracy_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_one_hot, y_pred_prob, multi_class="ovo", average='weighted')
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
#     G_mean = math.sqrt(recall * specificity)
    print(f" Fold {k + 1} | " )
    print(f" AUC_ROC: { roc_auc * 100}%" )
    print(f" ACC: { acc * 100}%" )
    print(f" F1: { f1 * 100}%" )
    print(f" RECALL: { recall * 100}%" )
    print(f" PRECISION: { precision * 100}%" )
    lr_f1_scores.append(f1)
    lr_roc_scores.append(roc_auc)
    lr_acc_scores.append(acc)
    lr_recall_scores.append(recall)
    lr_precision_scores.append(precision)
    
print(f'average roc score: {np.mean(lr_roc_scores)}')
print(f'average acc_score: {np.mean(lr_acc_scores)}')
print(f'average f1_score: {np.mean(lr_f1_scores)}')
print(f'average recall_score: {np.mean(lr_recall_scores)}')
print(f'average precision_score: {np.mean(lr_precision_scores)}')

第 1 折

 Fold 1 | 
 AUC_ROC: 65.38338689998376%
 ACC: 43.58373535492764%
 F1: 42.03328598324722%
 RECALL: 43.58373535492764%
 PRECISION: 43.147023227996215%
第 2 折

 Fold 2 | 
 AUC_ROC: 66.21304405342508%
 ACC: 44.05237767057202%
 F1: 42.26804122122232%
 RECALL: 44.05237767057202%
 PRECISION: 43.653998527107404%
第 3 折

 Fold 3 | 
 AUC_ROC: 65.6091745394617%
 ACC: 43.15644383184011%
 F1: 41.438983216711655%
 RECALL: 43.15644383184011%
 PRECISION: 42.62860266274343%
第 4 折

 Fold 4 | 
 AUC_ROC: 65.44250446619351%
 ACC: 43.6526533425224%
 F1: 41.9927230849236%
 RECALL: 43.6526533425224%
 PRECISION: 43.4107208733585%
第 5 折

 Fold 5 | 
 AUC_ROC: 66.01593909922421%
 ACC: 44.465885596140595%
 F1: 42.75058815330851%
 RECALL: 44.465885596140595%
 PRECISION: 44.24270234479827%
第 6 折

 Fold 6 | 
 AUC_ROC: 65.5165635042437%
 ACC: 43.128876636802204%
 F1: 41.45122944506042%
 RECALL: 43.128876636802204%
 PRECISION: 42.522021826635395%
第 7 折

 Fold 7 | 
 AUC_ROC: 65.57363534184171%
 ACC: 44.038594073053

In [7]:
data = pd.read_csv("data_stand.csv")
data.shape
X = data.drop(columns=['risk'], axis=1)
y = data['risk']
kf = KFold(n_splits=10, shuffle=True, random_state=0)
splits = kf.split(X, y)
next(iter(splits))

(array([    0,     1,     2, ..., 72547, 72548, 72549]),
 array([    3,    14,    18, ..., 72525, 72542, 72544]))

In [8]:
# adboost
from sklearn.ensemble import AdaBoostClassifier as ad

Ad_roc_scores = []
Ad_acc_scores = []
Ad_f1_scores = []
Ad_recall_scores = []
Ad_precision_scores = []
Ad_feature_importances = pd.DataFrame(index=None)
Ad_feature_importances['features'] = data.drop(['risk'], axis=1).columns

for k, (train_indices, test_indices) in enumerate(splits):
    print("第 %d 折\n" % (k + 1))
    X_train, X_test = X.iloc[train_indices], X.iloc[test_indices]
    y_train, y_test = y.iloc[train_indices], y.iloc[test_indices]
    
    labels = [0, 1, 2, 3]
    y_one_hot = label_binarize(y_test, classes=labels)
    Ad = ad(random_state=0)
    Ad.fit(X_train, y_train)
    Ad_feature_importances[f'fold_{k+1}'] = Ad.feature_importances_
    y_pred_prob = Ad.predict_proba(X_test)
    y_pred = Ad.predict(X_test)
    
    acc = accuracy_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_one_hot, y_pred_prob, multi_class="ovo", average='weighted')
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
#     G_mean = math.sqrt(recall * specificity)
    print(f" Fold {k + 1} | " )
    print(f" AUC_ROC: { roc_auc * 100}%" )
    print(f" ACC: { acc * 100}%" )
    print(f" F1: { f1 * 100}%" )
    print(f" RECALL: { recall * 100}%" )
    print(f" PRECISION: { precision * 100}%" )
    Ad_f1_scores.append(f1)
    Ad_roc_scores.append(roc_auc)
    Ad_acc_scores.append(acc)
    Ad_recall_scores.append(recall)
    Ad_precision_scores.append(precision)
    
print(f'average roc score: {np.mean(Ad_roc_scores)}')
print(f'average acc_score: {np.mean(Ad_acc_scores)}')
print(f'average f1_score: {np.mean(Ad_f1_scores)}')
print(f'average recall_score: {np.mean(Ad_recall_scores)}')
print(f'average precision_score: {np.mean(Ad_precision_scores)}')

第 1 折

 Fold 1 | 
 AUC_ROC: 77.89178464477648%
 ACC: 56.884906960716755%
 F1: 54.795443244186345%
 RECALL: 56.884906960716755%
 PRECISION: 55.06083700761432%
第 2 折

 Fold 2 | 
 AUC_ROC: 78.59422139353399%
 ACC: 58.345968297725705%
 F1: 56.626003141247175%
 RECALL: 58.345968297725705%
 PRECISION: 56.81102127014067%
第 3 折

 Fold 3 | 
 AUC_ROC: 78.40555980269852%
 ACC: 57.684355616815985%
 F1: 55.67134950910951%
 RECALL: 57.684355616815985%
 PRECISION: 55.974460193610284%
第 4 折

 Fold 4 | 
 AUC_ROC: 78.68971308217041%
 ACC: 57.87732598208132%
 F1: 55.86943966765158%
 RECALL: 57.87732598208132%
 PRECISION: 56.41081851174267%
第 5 折

 Fold 5 | 
 AUC_ROC: 78.45050780192625%
 ACC: 57.808407994486565%
 F1: 55.76411890939663%
 RECALL: 57.808407994486565%
 PRECISION: 55.74652955310974%
第 6 折

 Fold 6 | 
 AUC_ROC: 79.0534570890691%
 ACC: 58.11164713990351%
 F1: 56.07491204226645%
 RECALL: 58.11164713990351%
 PRECISION: 56.644734406861%
第 7 折

 Fold 7 | 
 AUC_ROC: 78.3324158732375%
 ACC: 57.4500344

In [9]:
data = pd.read_csv("data_stand.csv")
data.shape
X = data.drop(columns=['risk'], axis=1)
y = data['risk']
kf = KFold(n_splits=10, shuffle=True, random_state=0)
splits = kf.split(X, y)
next(iter(splits))

(array([    0,     1,     2, ..., 72547, 72548, 72549]),
 array([    3,    14,    18, ..., 72525, 72542, 72544]))

In [10]:
# DT

DT_roc_scores = []
DT_acc_scores = []
DT_f1_scores = []
DT_recall_scores = []
DT_precision_scores = []
DT_feature_importances = pd.DataFrame(index=None)
DT_feature_importances['features'] = data.drop(['risk'], axis=1).columns

for k, (train_indices, test_indices) in enumerate(splits):
    print("第 %d 折\n" % (k + 1))
    X_train, X_test = X.iloc[train_indices], X.iloc[test_indices]
    y_train, y_test = y.iloc[train_indices], y.iloc[test_indices]
    
    labels = [0, 1, 2, 3]
    y_one_hot = label_binarize(y_test, classes=labels)
    DT = DecisionTreeClassifier(random_state=0)
    DT.fit(X_train, y_train)
    DT_feature_importances[f'fold_{k+1}'] = DT.feature_importances_
    y_pred_prob = DT.predict_proba(X_test)
    y_pred = DT.predict(X_test)
    
    acc = accuracy_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_one_hot, y_pred_prob, multi_class="ovo", average='weighted')
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
#     G_mean = math.sqrt(recall * specificity)
    print(f" Fold {k + 1} | " )
    print(f" AUC_ROC: { roc_auc * 100}%" )
    print(f" ACC: { acc * 100}%" )
    print(f" F1: { f1 * 100}%" )
    print(f" RECALL: { recall * 100}%" )
    print(f" PRECISION: { precision * 100}%" )
    DT_f1_scores.append(f1)
    DT_roc_scores.append(roc_auc)
    DT_acc_scores.append(acc)
    DT_recall_scores.append(recall)
    DT_precision_scores.append(precision)
    
print(f'average roc score: {np.mean(DT_roc_scores)}')
print(f'average acc_score: {np.mean(DT_acc_scores)}')
print(f'average f1_score: {np.mean(DT_f1_scores)}')
print(f'average recall_score: {np.mean(DT_recall_scores)}')
print(f'average precision_score: {np.mean(DT_precision_scores)}')

第 1 折

 Fold 1 | 
 AUC_ROC: 71.23067459604434%
 ACC: 57.29841488628532%
 F1: 57.28101560579874%
 RECALL: 57.29841488628532%
 PRECISION: 57.302369245501175%
第 2 折

 Fold 2 | 
 AUC_ROC: 71.40752045175076%
 ACC: 57.47760165403171%
 F1: 57.46490773471502%
 RECALL: 57.47760165403171%
 PRECISION: 57.53294895850072%
第 3 折

 Fold 3 | 
 AUC_ROC: 71.65220177849196%
 ACC: 57.91867677463818%
 F1: 57.95691787720173%
 RECALL: 57.91867677463818%
 PRECISION: 58.0119698225435%
第 4 折

 Fold 4 | 
 AUC_ROC: 71.53491372858471%
 ACC: 57.7670572019297%
 F1: 57.74224139358377%
 RECALL: 57.7670572019297%
 PRECISION: 57.76221198073905%
第 5 折

 Fold 5 | 
 AUC_ROC: 71.59168940550187%
 ACC: 57.780840799448654%
 F1: 57.83083328171178%
 RECALL: 57.780840799448654%
 PRECISION: 57.884685690030224%
第 6 折

 Fold 6 | 
 AUC_ROC: 71.48153098881829%
 ACC: 57.83597518952447%
 F1: 57.91208039857521%
 RECALL: 57.83597518952447%
 PRECISION: 58.02627195655313%
第 7 折

 Fold 7 | 
 AUC_ROC: 71.07288738079379%
 ACC: 57.4224672639558

In [11]:
data = pd.read_csv("data_stand.csv")
data.shape
X = data.drop(columns=['risk'], axis=1)
y = data['risk']
kf = KFold(n_splits=10, shuffle=True, random_state=0)
splits = kf.split(X, y)
next(iter(splits))

(array([    0,     1,     2, ..., 72547, 72548, 72549]),
 array([    3,    14,    18, ..., 72525, 72542, 72544]))

In [12]:
# RF
from sklearn.ensemble import RandomForestClassifier

RF_roc_scores = []
RF_acc_scores = []
RF_f1_scores = []
RF_recall_scores = []
RF_precision_scores = []
RFC_feature_importances = pd.DataFrame(index=None)
RFC_feature_importances['features'] = data.drop(['risk'], axis=1).columns

for k, (train_indices, test_indices) in enumerate(splits):
    print("第 %d 折\n" % (k + 1))
    X_train, X_test = X.iloc[train_indices], X.iloc[test_indices]
    y_train, y_test = y.iloc[train_indices], y.iloc[test_indices]
    
    labels = [0, 1, 2, 3]
    y_one_hot = label_binarize(y_test, classes=labels)
    RFC = RandomForestClassifier(random_state=0)
    RFC.fit(X_train, y_train)
    RFC_feature_importances[f'fold_{k+1}'] = RFC.feature_importances_
    y_pred_prob = RFC.predict_proba(X_test)
    y_pred = RFC.predict(X_test)
    
    acc = accuracy_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_one_hot, y_pred_prob, multi_class="ovo", average='weighted')
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
#     G_mean = math.sqrt(recall * specificity)
    print(f" Fold {k + 1} | " )
    print(f" AUC_ROC: { roc_auc * 100}%" )
    print(f" ACC: { acc * 100}%" )
    print(f" F1: { f1 * 100}%" )
    print(f" RECALL: { recall * 100}%" )
    print(f" PRECISION: { precision * 100}%" )
    RF_f1_scores.append(f1)
    RF_roc_scores.append(roc_auc)
    RF_acc_scores.append(acc)
    RF_recall_scores.append(recall)
    RF_precision_scores.append(precision)
    
print(f'average roc score: {np.mean(RF_roc_scores)}')
print(f'average acc_score: {np.mean(RF_acc_scores)}')
print(f'average f1_score: {np.mean(RF_f1_scores)}')
print(f'average recall_score: {np.mean(RF_recall_scores)}')
print(f'average precision_score: {np.mean(RF_precision_scores)}')

第 1 折

 Fold 1 | 
 AUC_ROC: 84.68151507517089%
 ACC: 64.16264645072364%
 F1: 63.860661903506276%
 RECALL: 64.16264645072364%
 PRECISION: 63.791594359561785%
第 2 折

 Fold 2 | 
 AUC_ROC: 85.3678395461059%
 ACC: 65.3342522398346%
 F1: 64.9279405030068%
 RECALL: 65.3342522398346%
 PRECISION: 64.87446007146603%
第 3 折

 Fold 3 | 
 AUC_ROC: 85.18796178660932%
 ACC: 65.12749827705031%
 F1: 64.79481681009848%
 RECALL: 65.12749827705031%
 PRECISION: 64.7779158119392%
第 4 折

 Fold 4 | 
 AUC_ROC: 85.5688009115582%
 ACC: 65.47208821502413%
 F1: 65.04833273511821%
 RECALL: 65.47208821502413%
 PRECISION: 65.03671918354257%
第 5 折

 Fold 5 | 
 AUC_ROC: 85.46486044173969%
 ACC: 64.96209510682289%
 F1: 64.58040118733118%
 RECALL: 64.96209510682289%
 PRECISION: 64.54839850206703%
第 6 折

 Fold 6 | 
 AUC_ROC: 85.61135452102808%
 ACC: 65.63749138525155%
 F1: 65.36837745144155%
 RECALL: 65.63749138525155%
 PRECISION: 65.3967904872956%
第 7 折

 Fold 7 | 
 AUC_ROC: 84.64356020944665%
 ACC: 63.92832529290145%
 F1

In [13]:
data = pd.read_csv("data_stand.csv")
data.shape
X = data.drop(columns=['risk'], axis=1)
y = data['risk']
kf = KFold(n_splits=10, shuffle=True, random_state=0)
splits = kf.split(X, y)
next(iter(splits))

(array([    0,     1,     2, ..., 72547, 72548, 72549]),
 array([    3,    14,    18, ..., 72525, 72542, 72544]))

In [14]:
# xgboost
import xgboost as xgb

XGBR_roc_scores = []
XGBR_acc_scores = []
XGBR_f1_scores = []
XGBR_recall_scores = []
XGBR_precision_scores = []
xg_feature_importances = pd.DataFrame(index=None)
xg_feature_importances['features'] = data.drop(['risk'], axis=1).columns

for k, (train_indices, test_indices) in enumerate(splits):
    print("第 %d 折\n" % (k + 1))
    X_train, X_test = X.iloc[train_indices], X.iloc[test_indices]
    y_train, y_test = y.iloc[train_indices], y.iloc[test_indices]
    
    labels = [0, 1, 2, 3]
    y_one_hot = label_binarize(y_test, classes=labels)
    XGBR = xgb.XGBClassifier(random_state=0)
    XGBR.fit(X_train, y_train)
    xg_feature_importances[f'fold_{k+1}'] = XGBR.feature_importances_
    y_pred_prob = XGBR.predict_proba(X_test)
    y_pred = XGBR.predict(X_test)
    
    acc = accuracy_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_one_hot, y_pred_prob, multi_class="ovo", average='weighted')
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
#     G_mean = math.sqrt(recall * specificity)
    print(f" Fold {k + 1} | " )
    print(f" AUC_ROC: { roc_auc * 100}%" )
    print(f" ACC: { acc * 100}%" )
    print(f" F1: { f1 * 100}%" )
    print(f" RECALL: { recall * 100}%" )
    print(f" PRECISION: { precision * 100}%" )
    XGBR_f1_scores.append(f1)
    XGBR_roc_scores.append(roc_auc)
    XGBR_acc_scores.append(acc)
    XGBR_recall_scores.append(recall)
    XGBR_precision_scores.append(precision)
    
print(f'average roc score: {np.mean(XGBR_roc_scores)}')
print(f'average acc_score: {np.mean(XGBR_acc_scores)}')
print(f'average f1_score: {np.mean(XGBR_f1_scores)}')
print(f'average recall_score: {np.mean(XGBR_recall_scores)}')
print(f'average precision_score: {np.mean(XGBR_precision_scores)}')

第 1 折

 Fold 1 | 
 AUC_ROC: 86.17341784938093%
 ACC: 65.34803583735355%
 F1: 64.72631650289506%
 RECALL: 65.34803583735355%
 PRECISION: 64.80486430571918%
第 2 折

 Fold 2 | 
 AUC_ROC: 86.56155755613526%
 ACC: 66.58855961405928%
 F1: 66.07842686753574%
 RECALL: 66.58855961405928%
 PRECISION: 66.18124902006109%
第 3 折

 Fold 3 | 
 AUC_ROC: 86.65958662562078%
 ACC: 66.13370089593383%
 F1: 65.57853243242245%
 RECALL: 66.13370089593383%
 PRECISION: 65.66062768134458%
第 4 折

 Fold 4 | 
 AUC_ROC: 86.8225838770414%
 ACC: 66.97450034458994%
 F1: 66.33860275696945%
 RECALL: 66.97450034458994%
 PRECISION: 66.38871914439125%
第 5 折

 Fold 5 | 
 AUC_ROC: 86.56509456736768%
 ACC: 66.53342522398346%
 F1: 65.98242124099708%
 RECALL: 66.53342522398346%
 PRECISION: 66.0864140956266%
第 6 折

 Fold 6 | 
 AUC_ROC: 86.80093727606692%
 ACC: 66.91936595451413%
 F1: 66.5179518684157%
 RECALL: 66.91936595451413%
 PRECISION: 66.60670801190714%
第 7 折

 Fold 7 | 
 AUC_ROC: 86.22801401084153%
 ACC: 66.13370089593383%
 

In [15]:
data = pd.read_csv("data_stand.csv")
import re
data = data.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
data.to_csv('data_stand_lgb.csv', index=False, encoding='utf_8_sig')

In [16]:
X = data.drop(columns=['risk'], axis=1)
y = data['risk']
kf = KFold(n_splits=10, shuffle=True, random_state=0)
splits = kf.split(X, y)
next(iter(splits))

(array([    0,     1,     2, ..., 72547, 72548, 72549]),
 array([    3,    14,    18, ..., 72525, 72542, 72544]))

In [17]:
#lightGbm
import lightgbm as lgb

lgb_roc_scores = []
lgb_acc_scores = []
lgb_f1_scores = []
lgb_recall_scores = []
lgb_precision_scores = []
lgb_feature_importances = pd.DataFrame(index=None)
lgb_feature_importances['features'] = data.drop(['risk'], axis=1).columns

for k, (train_indices, test_indices) in enumerate(splits):
    print("第 %d 折\n" % (k + 1))
    X_train, X_test = X.iloc[train_indices], X.iloc[test_indices]
    y_train, y_test = y.iloc[train_indices], y.iloc[test_indices]
    
    labels = [0, 1, 2, 3]
    y_one_hot = label_binarize(y_test, classes=labels)
    
    LGB = lgb.LGBMClassifier(random_state=0)
    LGB.fit(X_train, y_train)
    lgb_feature_importances[f'fold_{k+1}'] = LGB.feature_importances_
    y_pred_prob = LGB.predict_proba(X_test)
    y_pred = LGB.predict(X_test)
    
    acc = accuracy_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_one_hot, y_pred_prob, multi_class="ovo", average='weighted')
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
#     G_mean = math.sqrt(recall * specificity)
    print(f" Fold {k + 1} | " )
    print(f" AUC_ROC: { roc_auc * 100}%" )
    print(f" ACC: { acc * 100}%" )
    print(f" F1: { f1 * 100}%" )
    print(f" RECALL: { recall * 100}%" )
    print(f" PRECISION: { precision * 100}%" )
    lgb_f1_scores.append(f1)
    lgb_roc_scores.append(roc_auc)
    lgb_acc_scores.append(acc)
    lgb_recall_scores.append(recall)
    lgb_precision_scores.append(precision)
    
print(f'average roc score: {np.mean(lgb_roc_scores)}')
print(f'average acc_score: {np.mean(lgb_acc_scores)}')
print(f'average f1_score: {np.mean(lgb_f1_scores)}')
print(f'average recall_score: {np.mean(lgb_recall_scores)}')
print(f'average precision_score: {np.mean(lgb_precision_scores)}')

第 1 折

 Fold 1 | 
 AUC_ROC: 85.87244729074189%
 ACC: 65.27911784975878%
 F1: 64.64200646882274%
 RECALL: 65.27911784975878%
 PRECISION: 64.69828650742986%
第 2 折

 Fold 2 | 
 AUC_ROC: 86.35114365064072%
 ACC: 66.2026188835286%
 F1: 65.62509451341634%
 RECALL: 66.2026188835286%
 PRECISION: 65.82922518915709%
第 3 折

 Fold 3 | 
 AUC_ROC: 86.38142611984615%
 ACC: 65.45830461750516%
 F1: 64.89964771613084%
 RECALL: 65.45830461750516%
 PRECISION: 64.98807836067732%
第 4 折

 Fold 4 | 
 AUC_ROC: 86.58783163449975%
 ACC: 66.28532046864231%
 F1: 65.57836583008707%
 RECALL: 66.28532046864231%
 PRECISION: 65.64516600049673%
第 5 折

 Fold 5 | 
 AUC_ROC: 86.21345492917209%
 ACC: 66.25775327360441%
 F1: 65.74812499954785%
 RECALL: 66.25775327360441%
 PRECISION: 65.82601449499349%
第 6 折

 Fold 6 | 
 AUC_ROC: 86.61071503649266%
 ACC: 66.4369400413508%
 F1: 65.95694198465685%
 RECALL: 66.4369400413508%
 PRECISION: 66.08652045385092%
第 7 折

 Fold 7 | 
 AUC_ROC: 85.9626090699593%
 ACC: 65.63749138525155%
 F1